In [13]:
# Spark init
!wget -q https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz 
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/default-java"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
!pip install -q findspark
import findspark
findspark.init()

In [14]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession

In [15]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Create spark session 
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [17]:
# Check raw data (reviews.csv)
!head -5 /content/drive/MyDrive/data/beers-breweries-reviews/reviews.csv

beer_id,username,date,text,look,smell,taste,feel,overall,score
271781,bluejacket74,2017-03-17,"   750 ml bottle, 2016 vintage, bottle #304 of 360. Served in a DFH snifter, the beer pours dark brown/black with about an inch tan head that stuck around a little while. There's also a good amount of lacing. I like the aroma, the brew smells like dark fruit (raisin, plum), chocolate, roasted malt, and a bit of licorice. For the most part I think the taste is just like the aroma, but with some coffee flavor noticeable too. Mouthfeel/body is medium/full, it's creamy and coating with a moderate amount of carbonation. I think it's a good overall brew, worth picking up if you want a nice straight forward imperial stout. Here's hoping Four String brings this back in the future! $9.99 a bottle.  ",4,4,4,4.25,4,4.03
125646,_dirty_,2017-12-21,  ,4.5,4.5,4.5,4.5,4.5,4.5
125646,CJDUBYA,2017-12-21,  ,4.75,4.75,4.75,4.75,4.75,4.75
125646,GratefulBeerGuy,2017-12-20,"   0% 16 oz can. Funny story: As I fina

In [21]:
# Read the reviews file and display top 20 rows
custom_schema = StructType([
        StructField("beer_id", IntegerType(), True),
        StructField("username", StringType(), True),
        StructField("date", DateType(), True),
        StructField("review_text", StringType(), True),
        StructField("look", FloatType(), False),
        StructField("smell", FloatType(), False),
        StructField("taste", FloatType(), False),
        StructField("feel", FloatType(), False),
        StructField("overall", FloatType(), False),
        StructField("score", FloatType(), False)
  ])

reviews_df = spark.read.csv('/content/drive/MyDrive/data/beers-breweries-reviews/reviews.csv',
                    sep = ',', schema = custom_schema, header = True)

reviews_df.show()

+-------+---------------+----------+--------------------+----+-----+-----+----+-------+-----+
|beer_id|       username|      date|         review_text|look|smell|taste|feel|overall|score|
+-------+---------------+----------+--------------------+----+-----+-----+----+-------+-----+
| 271781|   bluejacket74|2017-03-17|   750 ml bottle,...| 4.0|  4.0|  4.0|4.25|    4.0| 4.03|
| 125646|        _dirty_|2017-12-21|                    | 4.5|  4.5|  4.5| 4.5|    4.5|  4.5|
| 125646|        CJDUBYA|2017-12-21|                    |4.75| 4.75| 4.75|4.75|   4.75| 4.75|
| 125646|GratefulBeerGuy|2017-12-20|"   0% 16 oz can....|null| null| null|null|   null| null|
| 125646|       LukeGude|2017-12-20|   Classic TH NEI...|4.25|  4.5| 4.25|4.25|   4.25| 4.31|
| 125646|        jneff33|2017-12-20|                    | 4.5| 4.75|  5.0| 5.0|    5.0| 4.91|
| 125646|      yummybeer|2017-12-19|                    |4.75|  4.5| 4.75|4.75|   4.75| 4.69|
| 125646|           MFMB|2017-12-16|   Pours a creamy...|4.7

In [22]:
reviews_df.printSchema()

root
 |-- beer_id: integer (nullable = true)
 |-- username: string (nullable = true)
 |-- date: date (nullable = true)
 |-- review_text: string (nullable = true)
 |-- look: float (nullable = true)
 |-- smell: float (nullable = true)
 |-- taste: float (nullable = true)
 |-- feel: float (nullable = true)
 |-- overall: float (nullable = true)
 |-- score: float (nullable = true)



In [27]:
# Remove rows with missing values and create clean df
clean_reviews_df = reviews_df.dropna()

clean_reviews_df.show()

+-------+------------+----------+--------------------+----+-----+-----+----+-------+-----+
|beer_id|    username|      date|         review_text|look|smell|taste|feel|overall|score|
+-------+------------+----------+--------------------+----+-----+-----+----+-------+-----+
| 271781|bluejacket74|2017-03-17|   750 ml bottle,...| 4.0|  4.0|  4.0|4.25|    4.0| 4.03|
| 125646|     _dirty_|2017-12-21|                    | 4.5|  4.5|  4.5| 4.5|    4.5|  4.5|
| 125646|     CJDUBYA|2017-12-21|                    |4.75| 4.75| 4.75|4.75|   4.75| 4.75|
| 125646|    LukeGude|2017-12-20|   Classic TH NEI...|4.25|  4.5| 4.25|4.25|   4.25| 4.31|
| 125646|     jneff33|2017-12-20|                    | 4.5| 4.75|  5.0| 5.0|    5.0| 4.91|
| 125646|   yummybeer|2017-12-19|                    |4.75|  4.5| 4.75|4.75|   4.75| 4.69|
| 125646|        MFMB|2017-12-16|   Pours a creamy...|4.75|  4.5|  4.5| 4.5|    4.5| 4.52|
| 125646|        wwoj|2017-12-16|                    | 5.0| 4.75|  4.5| 4.5|    4.5| 4.59|

In [28]:
# Number of unique users
uniqueUsers = clean_reviews_df.select('username').distinct().count()
print("There are %d unique users that left a review." % uniqueUsers)

# Number of unique beers 
uniqueBeers = clean_reviews_df.select('beer_id').distinct().count()
print("There are %d beers that were reviewed." % uniqueBeers)

There are 108406 unique users that left a review.
There are 155199 beers that were reviewed.


In [34]:
# Find average ratings for all categories 
# This takes a long time to run (about 10 minutes)
import pyspark.sql.functions as func

averageLookRating = clean_reviews_df.agg(func.avg('look')).collect()[0][0]
print("The average rating for the Look category is: %.2f" % averageLookRating)

averageSmellRating = clean_reviews_df.agg(func.avg('smell')).collect()[0][0]
print("The average rating for the Smell category is: %.2f" % averageSmellRating) 

averageTasteRating = clean_reviews_df.agg(func.avg('taste')).collect()[0][0]
print("The average rating for the Taste category is: %.2f" % averageTasteRating) 

averageFeelRating = clean_reviews_df.agg(func.avg('feel')).collect()[0][0]
print("The average rating for the Feel category is: %.2f" % averageFeelRating) 

averageOverallRating = clean_reviews_df.agg(func.avg('overall')).collect()[0][0]
print("The average rating for the Overall category is: %.2f" % averageOverallRating)

averageScoreRating = clean_reviews_df.agg(func.avg('score')).collect()[0][0]
print("The average rating for the Score category is: %.2f" % averageScoreRating)

The average rating for the Look category is: 3.96
The average rating for the Smell category is: 3.89
The average rating for the Taste category is: 3.93
The average rating for the Feel category is: 3.89
The average rating for the Overall category is: 3.92
The average rating for the Score category is: 3.92


In [ ]:
# Check raw data (beers.csv)
!head -5 /content/drive/MyDrive/data/beers-breweries-reviews/beers.csv

id,name,brewery_id,state,country,style,availability,abv,notes,retired
202522,Olde Cogitator,2199,CA,US,English Oatmeal Stout, Rotating,7.3,No notes at this time.,f
82352,Konrads Stout Russian Imperial Stout,18604,,NO,Russian Imperial Stout, Rotating,10.4,No notes at this time.,f
214879,Scottish Right,44306,IN,US,Scottish Ale, Year-round,4,No notes at this time.,t
320009,MegaMeow Imperial Stout,4378,WA,US,American Imperial Stout, Winter,8.7,Every time this year,f


In [ ]:
# Read the beers file and display top 20 rows
beers_df = spark.read.csv('/content/drive/MyDrive/data/beers-breweries-reviews/beers.csv',
                    sep = ',', header = True)

beers_df.show()

+------+--------------------+----------+-----+-------+--------------------+--------------------+----+--------------------+-------+
|    id|                name|brewery_id|state|country|               style|        availability| abv|               notes|retired|
+------+--------------------+----------+-----+-------+--------------------+--------------------+----+--------------------+-------+
|202522|      Olde Cogitator|      2199|   CA|     US|English Oatmeal S...|            Rotating| 7.3|No notes at this ...|      f|
| 82352|Konrads Stout Rus...|     18604| null|     NO|Russian Imperial ...|            Rotating|10.4|No notes at this ...|      f|
|214879|      Scottish Right|     44306|   IN|     US|        Scottish Ale|          Year-round|   4|No notes at this ...|      t|
|320009|MegaMeow Imperial...|      4378|   WA|     US|American Imperial...|              Winter| 8.7|Every time this year|      f|
|246438|     Peaches-N-Cream|     44617|   PA|     US|  American Cream Ale|        

In [ ]:
# Ideas for cleaning the data:
  # converting each column to their appropriate datatype
  # address the null values in the state column

# Ideas for exploring/visualizing the data: 
  # what beer has the highest abv? Lowest? 
  # most popular beers in: Spring, Winter, Year-round 
  # visualize count of beers by country

In [ ]:
# Check raw data (breweries.csv)
!head -5 /content/drive/MyDrive/data/beers-breweries-reviews/breweries.csv

id,name,city,state,country,notes,types
19730,Brouwerij Danny,Erpe-Mere,,BE,No notes at this time.,Brewery
32541,Coachella Valley Brewing Co,Thousand Palms,CA,US,No notes at this time.,"Brewery, Bar, Beer-to-go"
44736,Beef 'O' Brady's,Plant City,FL,US,No notes at this time.,"Bar, Eatery"
23372,Broadway Wine Merchant,Oklahoma City,OK,US,No notes at this time.,Store


In [ ]:
# Read the breweries file and display top 20 rows
breweries_df = spark.read.csv('/content/drive/MyDrive/data/beers-breweries-reviews/breweries.csv',
                    sep = ',', header = True)

breweries_df.show()

+-----+--------------------+--------------------+-----+-------+--------------------+--------------------+
|   id|                name|                city|state|country|               notes|               types|
+-----+--------------------+--------------------+-----+-------+--------------------+--------------------+
|19730|     Brouwerij Danny|           Erpe-Mere| null|     BE|No notes at this ...|             Brewery|
|32541|Coachella Valley ...|      Thousand Palms|   CA|     US|No notes at this ...|Brewery, Bar, Bee...|
|44736|    Beef 'O' Brady's|          Plant City|   FL|     US|No notes at this ...|         Bar, Eatery|
|23372|Broadway Wine Mer...|       Oklahoma City|   OK|     US|No notes at this ...|               Store|
|35328|Brighton Beer Dis...|            Brighton|  GB2|     GB|Duplicate of http...|         Bar, Eatery|
|31561|      Teddy's Tavern|             Seattle|   WA|     US|No notes at this ...|     Bar, Beer-to-go|
|35975|Modus Operandi Br...|           Mona Va

In [ ]:
# Ideas for cleaning the data:
  # converting each column to their appropriate datatype
  # address the null values in the state column

# Ideas for exploring/visualizing the data: 
  # count unique breweries 
  # count unique types of breweries
  # visualize breweries by their type 
  # visualize breweries in the US by state 